In [27]:
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [28]:
# 1. 저장된 Sentence-BERT 임베딩 불러오기
# 저장된 임베딩 불러오기
with open('app_embeddings.pkl', 'rb') as file:
    app_embeddings = pickle.load(file)  # 임베딩 데이터를 불러옴

In [29]:
# 2. 새로운 데이터 Free_apps.csv 불러오기
apps_data = pd.read_csv('Free_apps.csv')


In [31]:
# 2. Sentence-BERT 모델 로드 및 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')
app_embeddings = model.encode(apps_data['Description'].values)


In [32]:
# 3. 생성된 임베딩을 pickle로 저장 (임시로 저장 가능)
with open('app_embeddings.pkl', 'wb') as file:
    pickle.dump(app_embeddings, file)


In [40]:
# 클러스터 수 범위 설정 (4부터 12까지)
for n_clusters in range(4, 13):
    print(f"\n\n==== {n_clusters} 클러스터에 대한 K-Means 클러스터링을 시작합니다 ====")

    # 4. 저장된 K-Means 모델 불러오기
    kmeans_model_filename = f'kmeans_model_{n_clusters}_clusters.pkl'
    try:
        with open(kmeans_model_filename, 'rb') as file:
            kmeans = pickle.load(file)  # 각 클러스터 수에 맞는 KMeans 모델 불러오기
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {kmeans_model_filename}")
        continue

    # 5. K-Means 클러스터링 수행
    new_clusters = kmeans.predict(app_embeddings)  # 임베딩을 기반으로 클러스터 예측

    # 클러스터 결과를 데이터에 추가
    apps_data['Cluster'] = new_clusters
    print(f"{n_clusters} 클러스터에 대한 클러스터링 완료")

    # 6. TF-IDF 기반으로 각 클러스터별 상위 키워드 추출
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(apps_data['Description'])

    # 클러스터별 상위 키워드 추출 함수
    def get_top_keywords(tfidf_matrix, clusters, top_n=5):
        num_clusters = np.unique(clusters).size
        cluster_centers = np.zeros((num_clusters, tfidf_matrix.shape[1]))

        # 클러스터별 중심 계산 (범위 체크 추가)
        for cluster in range(num_clusters):
            cluster_data = tfidf_matrix[clusters == cluster]
            if cluster_data.shape[0] > 0:  # 해당 클러스터에 데이터가 있는지 확인
                cluster_centers[cluster] = cluster_data.mean(axis=0)
            else:
                print(f"클러스터 {cluster}에 데이터가 없습니다. 건너뜁니다.")
        
        terms = vectorizer.get_feature_names_out()
        top_keywords = []

        # 각 클러스터의 상위 키워드 추출
        for cluster in range(num_clusters):
            center = cluster_centers[cluster]
            top_indices = center.argsort()[::-1][:top_n]
            keywords = [terms[i] for i in top_indices]
            top_keywords.append(keywords)

        return top_keywords

    # 7. 클러스터별 상위 5개 키워드 추출
    top_keywords_per_cluster = get_top_keywords(tfidf_matrix, new_clusters, top_n=5)

    # 8. Cosine Similarity를 사용하여 클러스터 대표 이름 결정
    def get_keyword_embedding(keywords):
        return model.encode(keywords)  # 상위 키워드를 다시 임베딩

    def select_representative_word(keywords):
        keyword_embeddings = get_keyword_embedding(keywords)
        similarity_matrix = cosine_similarity(keyword_embeddings)

        # 각 단어의 유사도 합 계산
        similarity_sums = similarity_matrix.sum(axis=1)

        # 유사도 합이 가장 큰 단어를 대표 단어로 선택
        representative_idx = np.argmax(similarity_sums)
        return keywords[representative_idx]

    # 9. 클러스터별 대표 단어(이름) 출력 및 추가
    cluster_names = []
    for cluster_num, keywords in enumerate(top_keywords_per_cluster):
        if len(keywords) > 0:  # 키워드가 존재하는 경우에만 처리
            category_name = select_representative_word(keywords)
            cluster_names.append(category_name)
            print(f"Cluster {cluster_num} 대표 이름: {category_name}")
        else:
            cluster_names.append(None)  # 데이터가 없는 클러스터에 대해 None 할당

    # 10. 클러스터별 대표 이름을 데이터에 추가 (범위를 벗어난 경우 처리)
    def assign_cluster_name(cluster_idx):
        if cluster_idx < len(cluster_names):
            return cluster_names[cluster_idx] if cluster_names[cluster_idx] is not None else "No Data"
        else:
            return "Out of Range"  # 만약 리스트 범위를 벗어났을 경우

    apps_data['Cluster Name'] = apps_data['Cluster'].apply(assign_cluster_name)

    # 11. 각 클러스터 결과를 CSV로 저장
    csv_filename = f'clustered_free_apps_{n_clusters}_clusters.csv'
    apps_data.to_csv(csv_filename, index=False)
    print(f"{n_clusters} 클러스터에 대한 클러스터링 결과가 '{csv_filename}'에 저장되었습니다.")



==== 4 클러스터에 대한 K-Means 클러스터링을 시작합니다 ====
4 클러스터에 대한 클러스터링 완료
Cluster 0 대표 이름: videos
Cluster 1 대표 이름: use
Cluster 2 대표 이름: tv
Cluster 3 대표 이름: friends
4 클러스터에 대한 클러스터링 결과가 'clustered_free_apps_4_clusters.csv'에 저장되었습니다.


==== 5 클러스터에 대한 K-Means 클러스터링을 시작합니다 ====
5 클러스터에 대한 클러스터링 완료
Cluster 0 대표 이름: app
Cluster 1 대표 이름: chats
Cluster 2 대표 이름: tv
Cluster 3 대표 이름: videos
Cluster 4 대표 이름: use
5 클러스터에 대한 클러스터링 결과가 'clustered_free_apps_5_clusters.csv'에 저장되었습니다.


==== 6 클러스터에 대한 K-Means 클러스터링을 시작합니다 ====
6 클러스터에 대한 클러스터링 완료
Cluster 0 대표 이름: mobile
Cluster 1 대표 이름: blood
Cluster 2 대표 이름: tv
Cluster 3 대표 이름: videos
Cluster 4 대표 이름: use
Cluster 5 대표 이름: game
6 클러스터에 대한 클러스터링 결과가 'clustered_free_apps_6_clusters.csv'에 저장되었습니다.


==== 7 클러스터에 대한 K-Means 클러스터링을 시작합니다 ====
7 클러스터에 대한 클러스터링 완료
Cluster 0 대표 이름: mobile
Cluster 1 대표 이름: movies
Cluster 2 대표 이름: use
Cluster 3 대표 이름: game
Cluster 4 대표 이름: videos
Cluster 5 대표 이름: reader
Cluster 6 대표 이름: blood
7 클러스터에 대한 클러스터링 결과가 'clustered_free_apps_7_c